# Implementación de pipelines de inferencia por lotes

La inferencia por lotes se utiliza en escenarios de producción con tareas de larga duración y grandes volúmenes de datos

**Inferencia por lotes**: Aplicación de un modelo predictivo a múltiples casos de forma asincrónica, generando resultados en un archivo o base de datos.

**Azure Machine Learning**: Plataforma para crear soluciones de inferencia por lotes.

Pipeline de inferencia por lotes:
- Lee datos de entrada.
- Carga un modelo registrado.
- Predice etiquetas.
- Escribe los resultados como salida.

Para crear una canalización de inferencia por lotes, Hay que realizar los siguiente:

**1. Registrar un modelo**

Para utilizar un modelo ya entrenado en un pipeline de inferencia por lotes, es necesario registrarlo en tu espacio de trabajo de Azure ML.

Si deseas registrar un modelo a partir de un archivo local, puedes hacer uso del método `register` del objeto Model.

In [ ]:
from azureml.core import Model

classification_model = Model.register(workspace=your_workspace,                         # registramos el modelo en el workspace
                                      model_name='classification_model',
                                      model_path='model.pkl',                           # ruta local del modelo
                                      description='A classification model')

Como alternativa, si tienes una referencia a la ejecución que se utilizó para entrenar el modelo, puedes hacer uso del método `register_model`.

In [ ]:
run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model')

**2. Crear un guión de puntuación**

El servicio de inferencia por lotes necesita un script de puntuación para cargar el modelo y usarlo para generar nuevas predicciones. Este script debe contener dos funciones:

- `init()`: Esta función se invoca cuando se inicializa el pipeline.
- `run(mini_batch)`: Esta función se invoca para cada lote de datos que se va a procesar.

Por lo general, la función `init` se utiliza para cargar el modelo desde el registro de modelos, y la función `run` se utiliza para generar predicciones a partir de cada lote de datos y devolver los resultados. 

In [ ]:
import os
import numpy as np
from azureml.core import Model
import joblib

def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

**3. Creación de una canalización con un ParallelRunStep**

Azure Machine Learning ofrece un tipo específico de paso de canalización, llamado `ParallelRunStep`, para realizar inferencias por lotes en paralelo. Esta clase permite leer lotes de archivos de un conjunto de datos de tipo `File` y escribir la salida del procesamiento en un `OutputFileDatasetConfig`.

Además, puedes configurar la opción `output_action` del paso a “append_row”. Esto asegura que todas las instancias del paso que se ejecutan en paralelo recopilen sus resultados en un único archivo de salida llamado *parallel_run_step.txt*.

In [ ]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core import Pipeline

# Get the batch dataset for input
batch_data_set = ws.datasets['batch-data']

# Set the output location
default_ds = ws.get_default_datastore()
output_dir = OutputFileDatasetConfig(name='inferences')

# Define the parallel run step step configuration
parallel_run_config = ParallelRunConfig(
    source_directory='batch_scripts',
    entry_script="batch_scoring_script.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=aml_cluster,
    node_count=4)

# Create the parallel run step
parallelrun_step = ParallelRunStep(
    name='batch-score',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('batch_data')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)
# Create the pipeline
pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])

**4. Ejecute la canalización y recupere el resultado del paso**

Una vez que hayas definido la canalización, puedes ejecutarla y esperar a que finalice. Posteriormente, puedes obtener el archivo *parallel_run_step.txt* de la salida del paso para revisar los resultados. 

In [ ]:
from azureml.core import Experiment

# Run the pipeline as an experiment
pipeline_run = Experiment(ws, 'batch_prediction_pipeline').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

# Get the outputs from the first (and only) step
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='results')

# Find the parallel_run_step.txt file
for root, dirs, files in os.walk('results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# Load and display the results
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]
print(df)

#### Publicar una canalización de inferencia por lotes

Podemos publicar un Pipeline de inferencia por lotes como un servicio REST

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(name='Batch_Prediction_Pipeline',
                                                   description='Batch pipeline',
                                                   version='1.0')
rest_endpoint = published_pipeline.endpoint

Una vez publicado, podemos usar el punto de conexión de servicio para iniciar el trabajo de inferencia por lotes

In [ ]:
import requests

response = requests.post(rest_endpoint,
                         headers=auth_header,
                         json={"ExperimentName": "Batch_Prediction"})
run_id = response.json()["Id"]

También podemos programar la canalización publicada para que se ejecute automáticamente

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

weekly = ScheduleRecurrence(frequency='Week', interval=1)
pipeline_schedule = Schedule.create(ws, name='Weekly Predictions',
                                        description='batch inferencing',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='Batch_Prediction',
                                        recurrence=weekly)